In [1]:
from cars_utils import make_training_dataset
import pandas as pd
import numpy as np
import sys

In [2]:
df_dia = pd.read_csv('tests/labeled/r_dia_k - 1 - Final.csv', index_col=0)
df_noche = pd.read_csv('tests/labeled/r_noche_k - Final.csv', index_col=0)

In [3]:
df_noche.frame_number = df_noche.frame_number + 73000
df = pd.concat([df_dia, df_noche], ignore_index=True)

In [4]:
df[['x_centroid', 'y_centroid']] = df.yolo_centroid.str[1:-1].str.split(',', expand=True).astype('float')

In [5]:
df = df.drop(columns=['vehicle_type', 'yolo_confidence', 'yolo_centroid'])

In [6]:
df.head()

,frame_number,yolo_rect_area,b_mean,g_mean,r_mean,plate,plate_confidence,Label,Numero carro,Talanquera,x_centroid,y_centroid
0,1,242606,86.896787,87.765991,102.278505,NaN,NaN,1.0,1.0,1.0,673.5,266.0
1,2,239328,86.780690,87.449471,102.158180,NaN,NaN,1.0,1.0,1.0,674.0,266.0
2,3,240929,86.756933,87.457608,102.179792,NaN,NaN,1.0,1.0,1.0,673.5,266.5
3,4,241791,86.973043,87.615283,102.400023,ZZL578,90.968361,1.0,1.0,1.0,673.5,266.5
4,5,240108,86.765860,87.351346,102.312597,NaN,NaN,1.0,1.0,1.0,673.5,267.0


In [7]:
data_df = make_training_dataset(df, windows=[1,3,10,25,50])

In [8]:
data_df.columns

Index(['ID', 'frame_number_1', 'yolo_rect_area_1', 'b_mean_1', 'g_mean_1',
       'r_mean_1', 'x_centroid_1', 'y_centroid_1', 'frame_number_2',
       'yolo_rect_area_2', 'b_mean_2', 'g_mean_2', 'r_mean_2', 'x_centroid_2',
       'y_centroid_2', 'different_cars'],
      dtype='object')

In [9]:
data_df.head()

,ID,frame_number_1,yolo_rect_area_1,b_mean_1,g_mean_1,r_mean_1,x_centroid_1,y_centroid_1,frame_number_2,yolo_rect_area_2,b_mean_2,g_mean_2,r_mean_2,x_centroid_2,y_centroid_2,different_cars
0,0_1,1.0,242606.0,86.896787,87.765991,102.278505,673.5,266.0,2.0,239328.0,86.780690,87.449471,102.158180,674.0,266.0,0
1,0_3,1.0,242606.0,86.896787,87.765991,102.278505,673.5,266.0,4.0,241791.0,86.973043,87.615283,102.400023,673.5,266.5,0
2,0_10,1.0,242606.0,86.896787,87.765991,102.278505,673.5,266.0,11.0,264306.0,92.186829,91.370476,108.528603,672.5,269.0,0
3,0_25,1.0,242606.0,86.896787,87.765991,102.278505,673.5,266.0,26.0,470586.0,92.384321,91.661624,110.075699,562.5,345.0,0
4,0_50,1.0,242606.0,86.896787,87.765991,102.278505,673.5,266.0,216.0,82110.0,98.855840,94.014724,89.656741,770.0,185.5,1


In [10]:
data_df = data_df[['yolo_rect_area_1', 'b_mean_1', 'g_mean_1', 'r_mean_1', 
                   'x_centroid_1', 'y_centroid_1','yolo_rect_area_2', 'b_mean_2', 
                   'g_mean_2', 'r_mean_2', 'x_centroid_2', 'y_centroid_2', 
                   'different_cars']]

In [11]:
data_df.different_cars.value_counts()

0    225760
1     20940
Name: different_cars, dtype: int64

In [12]:
from sklearn.model_selection import StratifiedShuffleSplit

split = StratifiedShuffleSplit(n_splits=1, test_size=0.20, random_state=42)

for train_index, test_index in split.split(data_df, data_df["different_cars"]):
    train_set = data_df.loc[train_index]
    test_set = data_df.loc[test_index]

In [13]:
train_set.different_cars.value_counts()

0    180608
1     16752
Name: different_cars, dtype: int64

In [14]:
test_set.different_cars.value_counts()

0    45152
1     4188
Name: different_cars, dtype: int64

In [15]:
X_train_df = train_set.drop("different_cars", axis=1)
X_test_df = test_set.drop("different_cars", axis=1)
y_train_df = train_set["different_cars"].copy()
y_test_df = test_set["different_cars"].copy()

In [16]:
X_train = X_train_df.to_numpy()
X_test = X_test_df.to_numpy()
y_train = y_train_df.to_numpy()
y_test = y_test_df.to_numpy()

In [17]:
X = np.concatenate((X_train, X_test), axis=0)
y = np.concatenate((y_train, y_test), axis=0)

In [18]:
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict

from sklearn.metrics import roc_auc_score, precision_score, recall_score

### Stochastic Gradient Descent

In [19]:
sgd_clf = SGDClassifier(random_state=42, class_weight="balanced")
y_pred_sgd = cross_val_predict(sgd_clf, X_train, y_train, cv=3)
cross_val_score(sgd_clf, X_train, y_train, cv=3, scoring="accuracy")
y_pred_sgd_for_roc = cross_val_predict(sgd_clf, X_train, y_train, cv=3, method="decision_function")
roc_auc_score_sgd = roc_auc_score(y_train, y_pred_sgd_for_roc)
roc_auc_score_sgd

0.8833072088518409

### Random Forest

In [20]:
forest_clf = RandomForestClassifier(random_state=42, n_jobs=-1)#class_weight="balanced"
cross_val_score(forest_clf, X_train, y_train, cv=3, scoring="recall")
y_probas_forest =  cross_val_predict(forest_clf, X_train, y_train, cv=3, method="predict_proba")
y_pred_forest = y_probas_forest[:, 1]
roc_auc_score_forest = roc_auc_score(y_train, y_pred_forest)
roc_auc_score_forest

/home/santiago/anaconda3/envs/cars/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/home/santiago/anaconda3/envs/cars/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/home/santiago/anaconda3/envs/cars/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/home/santiago/anaconda3/envs/cars/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarni

0.9969958246031382

### Decision Tree

In [21]:
dtree_clf = DecisionTreeClassifier(max_depth=6, random_state=42)#class_weight="balanced"
cross_val_score(dtree_clf, X_train, y_train, cv=3, scoring="recall")
y_probas_dtree = cross_val_predict(dtree_clf, X_train, y_train, cv=3, method="predict_proba")
y_pred_dtree = y_probas_dtree[:, 1]
roc_auc_score_dtree = roc_auc_score(y_train, y_pred_dtree)
roc_auc_score_dtree

0.9432787847319352

### LogReg

In [22]:
log_clf = LogisticRegression(random_state=42, n_jobs=-1)#class_weight="balanced"
cross_val_score(log_clf, X_train, y_train, cv=3, scoring="recall")
y_probas_log = cross_val_predict(log_clf, X_train, y_train, cv=3, method="predict_proba")
y_pred_log = y_probas_log[:, 1]
roc_auc_score_log = roc_auc_score(y_train, y_pred_log)
roc_auc_score_log

/home/santiago/anaconda3/envs/cars/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/santiago/anaconda3/envs/cars/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:1544: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 12.
  " = {}.".format(effective_n_jobs(self.n_jobs)))
/home/santiago/anaconda3/envs/cars/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/santiago/anaconda3/envs/cars/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:1544: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 12.
  " = {}.".format(effective_n_jobs(self.n_jobs)))
/home/santiago/anaconda3/envs/

0.9504880541173839

### SVM

In [23]:
#svm_clf = SVC(random_state=42)#class_weight="balanced",
#cross_val_score(svm_clf, X_train, y_train, cv=3, scoring="recall")
#y_pred_svm = cross_val_predict(svm_clf, X_train, y_train, cv=3)
#roc_auc_score_svm = roc_auc_score(y_train, y_pred_svm)
#roc_auc_score_svm

### Voting

In [24]:
voting_clf = VotingClassifier(
    estimators=[("lr", log_clf), ("rf", forest_clf), ("sgd", sgd_clf), ("dtree", dtree_clf)],
    voting = "hard"
    )
cross_val_score(voting_clf, X_train, y_train, cv=3, scoring="recall")
y_pred_voting = cross_val_predict(voting_clf, X_train, y_train, cv=3)
roc_auc_score_voting = roc_auc_score(y_train, y_pred_voting)
roc_auc_score_voting

print("Model scores after cross_validation:")
print("    -SGD: " + str(roc_auc_score_sgd))
print("    -RForest: " + str(roc_auc_score_forest))
print("    -DTree: " + str(roc_auc_score_dtree))
print("    -Voting: " + str(roc_auc_score_voting))
print("    -Log: " + str(roc_auc_score_log))

/home/santiago/anaconda3/envs/cars/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/santiago/anaconda3/envs/cars/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:1544: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 12.
  " = {}.".format(effective_n_jobs(self.n_jobs)))
/home/santiago/anaconda3/envs/cars/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/home/santiago/anaconda3/envs/cars/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/santiago/anaconda3/envs/cars/lib

Model scores after cross_validation:
    -SGD: 0.8833072088518409
    -RForest: 0.9969958246031382
    -DTree: 0.9432787847319352
    -Voting: 0.8760201691859296
    -Log: 0.9504880541173839


In [25]:
from sklearn.model_selection import GridSearchCV

param_grid_forest = [{"n_estimators":[1,2,3,4,5,15,25], 
                      "max_depth":[1,2,3,4,5,10,15], 
                      "criterion":["gini","entropy"], 
                      "class_weight":["balanced", None]}]

In [26]:
grid_search_forest =  GridSearchCV(forest_clf, param_grid_forest, cv=3, scoring="roc_auc", n_jobs=-1)
grid_search_forest.fit(X_train, y_train)

GridSearchCV(cv=3, error_score='raise-deprecating',
             estimator=RandomForestClassifier(bootstrap=True, class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features='auto',
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              n_estimators='warn', n_jobs=-1,
                                              oob_score=False, random_state=42,
                                              verbose=0, warm_start=False),
             iid='warn', n_jobs=-1,
             param_grid=[{'cla

In [27]:
print(grid_search_forest.best_estimator_)
print(grid_search_forest.best_score_)
print(grid_search_forest.best_params_)

RandomForestClassifier(bootstrap=True, class_weight='balanced',
                       criterion='entropy', max_depth=15, max_features='auto',
                       max_leaf_nodes=None, min_impurity_decrease=0.0,
                       min_impurity_split=None, min_samples_leaf=1,
                       min_samples_split=2, min_weight_fraction_leaf=0.0,
                       n_estimators=25, n_jobs=-1, oob_score=False,
                       random_state=42, verbose=0, warm_start=False)
0.9989847831340436
{'class_weight': 'balanced', 'criterion': 'entropy', 'max_depth': 15, 'n_estimators': 25}


In [28]:
forest_model = grid_search_forest.best_estimator_

In [29]:
forest_model.fit(X_train, y_train)
print(forest_model.score(X_test, y_test))

0.9970409404134576


In [30]:
forest_pred = forest_model.predict(X_test)

In [31]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, forest_pred)

array([[45104,    48],
       [   98,  4090]])

In [32]:
forest_model.fit(X, y)
forest_pred = forest_model.predict(X)

In [33]:
confusion_matrix(y, forest_pred)

array([[225704,     56],
       [   132,  20808]])

In [34]:
import pickle

with open('forest.pickle', 'wb') as f:
    pickle.dump(forest_model, f)